In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/subj-full-dataset/subj_dataset.xlsx
/kaggle/input/fasttext-bengali-word-embeddings/cc.bn.300.bin
/kaggle/input/glove-embeddings/glove.6B.300d.txt
/kaggle/input/glove-embeddings/glove.6B.100d.txt
/kaggle/input/bengaliword2vec/bn_w2v_model.text
/kaggle/input/bengaliword2vec/bn_w2v_model.bin
/kaggle/input/banglafont/kalpurush.ttf
/kaggle/input/subjectivity-detection-splitted/Split 70/test.xlsx
/kaggle/input/subjectivity-detection-splitted/Split 70/train.xlsx
/kaggle/input/subjectivity-detection-splitted/Split 70/val.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/test.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/train.xlsx
/kaggle/input/subjectivity-detection-splitted/Split80/val.xlsx
/kaggle/input/bengali-stopwords/bangla_stopwords.txt


In [2]:
#config
RANDOM_SEED = 42
import random
import os
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # Slower but reproducible
    torch.backends.cudnn.benchmark = False     # Disable benchmarking to ensure determinism

    # Set seed in Transformers (for Trainer)
    from transformers import set_seed as hf_set_seed
    hf_set_seed(seed)

set_seed(42)

2025-09-25 11:15:26.983656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758798927.166914      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758798927.223088      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_df = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/train.xlsx", engine='openpyxl')
val_df   = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/val.xlsx", engine='openpyxl')
test_df  = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/test.xlsx", engine='openpyxl')


In [4]:
df_full=pd.read_excel('/kaggle/input/subj-full-dataset/subj_dataset.xlsx', engine='openpyxl')
df_full.columns
print(df_full.shape)

(8655, 9)


In [20]:
#config
RANDOM_SEED = 42
MAX_FEATURES = 10000
MAX_ITER = 10000

In [22]:
#import 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def encode_labels(train_df, val_df, test_df):
    """
    Encodes the 'Label' column in train, val, and test DataFrames using LabelEncoder.
    Returns the updated DataFrames.
    """
    le = LabelEncoder()
    train_df['Label'] = le.fit_transform(train_df['Label'])
    val_df['Label']   = le.transform(val_df['Label'])
    test_df['Label']  = le.transform(test_df['Label'])
    return train_df, val_df, test_df


def prepare_datasets(train_df, val_df, test_df, transformerDL=True):
    """
    Extracts features and labels from the given DataFrames based on transformerDL flag.
    Returns: X_train, X_val, X_test, y_train, y_val, y_test
    """
    text_col = 'Text' if transformerDL else 'cleanedText'
    
    X_train = train_df[text_col]
    y_train = train_df['Label']
    
    X_val = val_df[text_col]
    y_val = val_df['Label']
    
    X_test = test_df[text_col]
    y_test = test_df['Label']
    
    return X_train, X_val, X_test, y_train, y_val, y_test
    
# Encode labels
train_df, val_df, test_df = encode_labels(train_df, val_df, test_df)

# Prepare datasets
#X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(train_df, val_df, test_df, transformerDL=True)


In [24]:
def trainDL():
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Conv1D, MaxPooling1D, Dropout
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
    from gensim.models import KeyedVectors
    from gensim.models.fasttext import load_facebook_vectors
    from gensim.models import Word2Vec
    from tensorflow.keras.layers import BatchNormalization
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.optimizers import Adam, RMSprop
    import itertools
    import time
    
    # Hyperparameter ranges
    HYPERPARAMS = {
        'max_vocab': [5000, 10000, 15000],
        'max_len': [64, 128, 256],
        'embedding_dim': [100, 200, 300],
        'lstm_units': [64, 128, 256],
        'dense_units': [32, 64, 128],
        'dropout_rates': [(0.3, 0.2), (0.5, 0.3), (0.6, 0.4)],  # (lstm_dropout, dense_dropout)
        'batch_size': [32, 64, 128],
        'learning_rate': [0.001, 0.01, 0.0001],
        'optimizer': ['adam', 'rmsprop'],
        'l2_reg': [0.0, 0.001, 0.01],
        'conv_filters': [64, 128, 256],
        'conv_kernel': [3, 5, 7],
        'trainable_embedding': [True, False]
    }
    
    def generate_hyperparameter_combinations(limit=20):
        """Generate a limited number of hyperparameter combinations"""
        all_combinations = []
        
        # Basic combinations for each model type
        basic_configs = [
            # Fast training configs
            {
                'max_vocab': 10000, 'max_len': 128, 'embedding_dim': 300,
                'lstm_units': 64, 'dense_units': 32, 'dropout_rates': (0.3, 0.2),
                'batch_size': 64, 'learning_rate': 0.001, 'optimizer': 'adam',
                'l2_reg': 0.0, 'conv_filters': 64, 'conv_kernel': 3, 'trainable_embedding': True
            },
            # Medium configs
            {
                'max_vocab': 10000, 'max_len': 128, 'embedding_dim': 300,
                'lstm_units': 128, 'dense_units': 64, 'dropout_rates': (0.5, 0.3),
                'batch_size': 64, 'learning_rate': 0.001, 'optimizer': 'adam',
                'l2_reg': 0.001, 'conv_filters': 128, 'conv_kernel': 5, 'trainable_embedding': True
            },
            # High capacity configs
            {
                'max_vocab': 15000, 'max_len': 256, 'embedding_dim': 300,
                'lstm_units': 256, 'dense_units': 128, 'dropout_rates': (0.6, 0.4),
                'batch_size': 32, 'learning_rate': 0.0001, 'optimizer': 'adam',
                'l2_reg': 0.01, 'conv_filters': 256, 'conv_kernel': 7, 'trainable_embedding': False
            },
            # Alternative optimizer configs
            {
                'max_vocab': 10000, 'max_len': 128, 'embedding_dim': 200,
                'lstm_units': 128, 'dense_units': 64, 'dropout_rates': (0.4, 0.3),
                'batch_size': 128, 'learning_rate': 0.01, 'optimizer': 'rmsprop',
                'l2_reg': 0.001, 'conv_filters': 128, 'conv_kernel': 5, 'trainable_embedding': True
            }
        ]
        
        # some random combinations
        import random
        for _ in range(limit - len(basic_configs)):
            config = {
                'max_vocab': random.choice(HYPERPARAMS['max_vocab']),
                'max_len': random.choice(HYPERPARAMS['max_len']),
                'embedding_dim': random.choice(HYPERPARAMS['embedding_dim']),
                'lstm_units': random.choice(HYPERPARAMS['lstm_units']),
                'dense_units': random.choice(HYPERPARAMS['dense_units']),
                'dropout_rates': random.choice(HYPERPARAMS['dropout_rates']),
                'batch_size': random.choice(HYPERPARAMS['batch_size']),
                'learning_rate': random.choice(HYPERPARAMS['learning_rate']),
                'optimizer': random.choice(HYPERPARAMS['optimizer']),
                'l2_reg': random.choice(HYPERPARAMS['l2_reg']),
                'conv_filters': random.choice(HYPERPARAMS['conv_filters']),
                'conv_kernel': random.choice(HYPERPARAMS['conv_kernel']),
                'trainable_embedding': random.choice(HYPERPARAMS['trainable_embedding'])
            }
            basic_configs.append(config)
        
        return basic_configs

    # GloVe loading function
    def load_glove_embeddings(glove_file):
        embeddings_index = {}
        with open(glove_file, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
        return embeddings_index
    
    # Generic Embedding Matrix Builder
    def build_embedding_matrix(word_index, word_vectors, max_vocab, embedding_dim):
        embedding_matrix = np.zeros((max_vocab, embedding_dim))
        for word, i in word_index.items():
            if i < max_vocab and word in word_vectors:
                if len(word_vectors[word]) == embedding_dim:
                    embedding_matrix[i] = word_vectors[word]
        return embedding_matrix
    
    # Model creation functions with hyperparameters
    def create_lstm_model(embedding_matrix, config):
        model = Sequential([
            Embedding(input_dim=config['max_vocab'], output_dim=config['embedding_dim'],
                     weights=[embedding_matrix], input_length=config['max_len'], 
                     trainable=config['trainable_embedding']),
            LSTM(config['lstm_units'], kernel_regularizer=l2(config['l2_reg'])),
            Dropout(config['dropout_rates'][0]),
            Dense(config['dense_units'], activation='relu', kernel_regularizer=l2(config['l2_reg'])),
            Dropout(config['dropout_rates'][1]),
            Dense(1, activation='sigmoid')
        ])
        
        optimizer = Adam(learning_rate=config['learning_rate']) if config['optimizer'] == 'adam' else RMSprop(learning_rate=config['learning_rate'])
        model.compile(loss='binary_crossentropy', optimizer=optimizer,
                     metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        return model
    
    def create_bilstm_model(embedding_matrix, config):
        model = Sequential([
            Embedding(input_dim=config['max_vocab'], output_dim=config['embedding_dim'],
                     weights=[embedding_matrix], input_length=config['max_len'], 
                     trainable=config['trainable_embedding']),
            Bidirectional(LSTM(config['lstm_units'], kernel_regularizer=l2(config['l2_reg']))),
            Dropout(config['dropout_rates'][0]),
            Dense(config['dense_units'], activation='relu', kernel_regularizer=l2(config['l2_reg'])),
            Dropout(config['dropout_rates'][1]),
            Dense(1, activation='sigmoid')
        ])
        
        optimizer = Adam(learning_rate=config['learning_rate']) if config['optimizer'] == 'adam' else RMSprop(learning_rate=config['learning_rate'])
        model.compile(loss='binary_crossentropy', optimizer=optimizer,
                     metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        return model
    
    def create_cnn_bilstm_model(embedding_matrix, config):
        model = Sequential([
            Embedding(input_dim=config['max_vocab'], output_dim=config['embedding_dim'],
                     weights=[embedding_matrix], input_length=config['max_len'], 
                     trainable=config['trainable_embedding']),
            Conv1D(config['conv_filters'], kernel_size=config['conv_kernel'], 
                   activation='relu', kernel_regularizer=l2(config['l2_reg'])),
            MaxPooling1D(pool_size=2),
            Bidirectional(LSTM(config['lstm_units'], kernel_regularizer=l2(config['l2_reg']))),
            Dropout(config['dropout_rates'][0]),
            Dense(config['dense_units'], activation='relu', kernel_regularizer=l2(config['l2_reg'])),
            Dropout(config['dropout_rates'][1]),
            Dense(1, activation='sigmoid')
        ])
        
        optimizer = Adam(learning_rate=config['learning_rate']) if config['optimizer'] == 'adam' else RMSprop(learning_rate=config['learning_rate'])
        model.compile(loss='binary_crossentropy', optimizer=optimizer,
                     metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        return model
    
    # training loop
    print("hyperparameter tuning...")
    
    # Generate hyperparameter combinations
    hp_combinations = generate_hyperparameter_combinations(limit=12)  # Adjust limit as needed
    
    # Load embeddings once
    print("Loading embeddings...")
    glove_vectors = load_glove_embeddings('/kaggle/input/glove-embeddings/glove.6B.300d.txt')
    fasttext_model = load_facebook_vectors('/kaggle/input/fasttext-bengali-word-embeddings/cc.bn.300.bin')
    word2vec_model = KeyedVectors.load_word2vec_format('/kaggle/input/bengaliword2vec/bn_w2v_model.text', binary=False)
    
    embedding_models = {
        "GloVe": glove_vectors,
        "FastText": fasttext_model,
        "Word2Vec": word2vec_model
    }
    
    model_builders = {
        "LSTM": create_lstm_model,
        "BiLSTM": create_bilstm_model,
        "CNN + BiLSTM": create_cnn_bilstm_model
    }
    
    # Get dataset
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(train_df, val_df, test_df, transformerDL=True)
    
    all_results = []
    total_experiments = len(hp_combinations) * len(embedding_models) * len(model_builders)
    experiment_count = 0
    
    for config_idx, config in enumerate(hp_combinations):
        print(f"\n{'='*60}")
        print(f"CONFIGURATION {config_idx + 1}/{len(hp_combinations)}")
        print(f"Config: {config}")
        print(f"{'='*60}")
        
        # Tokenize with current vocab size
        tokenizer = Tokenizer(num_words=config['max_vocab'])
        tokenizer.fit_on_texts(X_train)
        
        X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=config['max_len'])
        X_val_pad = pad_sequences(tokenizer.texts_to_sequences(X_val), maxlen=config['max_len'])
        X_test_pad = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=config['max_len'])
        
        # Create embedding matrices for current config
        embedding_matrices = {}
        for embed_name, embed_model in embedding_models.items():
            embedding_matrices[embed_name] = build_embedding_matrix(
                tokenizer.word_index, embed_model, config['max_vocab'], config['embedding_dim']
            )
        
        for embed_name, embed_matrix in embedding_matrices.items():
            for model_name, model_fn in model_builders.items():
                experiment_count += 1
                print(f"\n[{experiment_count}/{total_experiments}] {embed_name} + {model_name}")
                
                try:
                    start_time = time.time()
                    
                    # Create model
                    model = model_fn(embed_matrix, config)
                    
                    # Callbacks
                    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
                    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
                    
                    # Train
                    history = model.fit(
                        X_train_pad, y_train,
                        epochs=20,  
                        batch_size=config['batch_size'],
                        validation_data=(X_val_pad, y_val),
                        callbacks=[early_stop, reduce_lr],
                        verbose=1
                    )
                    
                    # Predict and evaluate
                    y_pred = (model.predict(X_test_pad, batch_size=config['batch_size']) > 0.5).astype("int32")
                    
                    acc = accuracy_score(y_test, y_pred)
                    prec = precision_score(y_test, y_pred, zero_division=0)
                    rec = recall_score(y_test, y_pred, zero_division=0)
                    f1 = f1_score(y_test, y_pred, zero_division=0)
                    
                    training_time = time.time() - start_time
                    
                    result = {
                        "Config_ID": config_idx,
                        "Embedding": embed_name,
                        "Model": model_name,
                        "Max_Vocab": config['max_vocab'],
                        "Max_Len": config['max_len'],
                        "Embedding_Dim": config['embedding_dim'],
                        "LSTM_Units": config['lstm_units'],
                        "Dense_Units": config['dense_units'],
                        "Dropout_LSTM": config['dropout_rates'][0],
                        "Dropout_Dense": config['dropout_rates'][1],
                        "Batch_Size": config['batch_size'],
                        "Learning_Rate": config['learning_rate'],
                        "Optimizer": config['optimizer'],
                        "L2_Reg": config['l2_reg'],
                        "Conv_Filters": config['conv_filters'] if 'CNN' in model_name else None,
                        "Conv_Kernel": config['conv_kernel'] if 'CNN' in model_name else None,
                        "Trainable_Embedding": config['trainable_embedding'],
                        "Accuracy": acc,
                        "Precision": prec,
                        "Recall": rec,
                        "F1_Score": f1,
                        "Training_Time": training_time,
                        "Epochs_Trained": len(history.history['loss'])
                    }
                    
                    all_results.append(result)
                    
                    print(f"Acc: {acc:.4f}, F1: {f1:.4f}, Time: {training_time:.1f}s")
                    
                    # Clean up
                    del model
                    tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error: {str(e)}")
                    continue
    
    # Results analysis
    results_df = pd.DataFrame(all_results)
    
    if not results_df.empty:
        print("\n" + "="*80)
        print("TOP 10 RESULTS BY F1 SCORE")
        print("="*80)
        
        top_results = results_df.nlargest(10, 'F1_Score')
        print(top_results[['Embedding', 'Model', 'Max_Vocab', 'LSTM_Units', 'Learning_Rate', 
                          'Batch_Size', 'F1_Score', 'Accuracy', 'Training_Time']].to_string(index=False))
        
        print(f"\n SUMMARY STATISTICS")
        print(f"Total experiments completed: {len(results_df)}")
        print(f"Best F1 Score: {results_df['F1_Score'].max():.4f}")
        print(f"Average F1 Score: {results_df['F1_Score'].mean():.4f}")
        print(f"Best combination: {results_df.loc[results_df['F1_Score'].idxmax()]['Embedding']} + {results_df.loc[results_df['F1_Score'].idxmax()]['Model']}")
        
        # Save detailed results
        results_df.to_csv('hyperparameter_tuning_results.csv', index=False)
        print(f"Results saved to 'hyperparameter_tuning_results.csv'")
        
        # Best hyperparameters
        best_config = results_df.loc[results_df['F1_Score'].idxmax()]
        print(f"\n BEST HYPERPARAMETERS:")
        for col in results_df.columns:
            if col not in ['Config_ID', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Training_Time', 'Epochs_Trained']:
                print(f"{col}: {best_config[col]}")
    else:
        print(" No successful experiments completed")
    
    return results_df

# Run the training
trainDL()

hyperparameter tuning...
Loading embeddings...

CONFIGURATION 1/12
Config: {'max_vocab': 10000, 'max_len': 128, 'embedding_dim': 300, 'lstm_units': 64, 'dense_units': 32, 'dropout_rates': (0.3, 0.2), 'batch_size': 64, 'learning_rate': 0.001, 'optimizer': 'adam', 'l2_reg': 0.0, 'conv_filters': 64, 'conv_kernel': 3, 'trainable_embedding': True}

[1/108] GloVe + LSTM


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1758799180.362899      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/20


I0000 00:00:1758799186.461266      62 cuda_dnn.cc:529] Loaded cuDNN version 90300


109/109 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.6758 - loss: 0.5855 - precision: 0.6991 - recall: 0.3663 - val_accuracy: 0.8116 - val_loss: 0.4163 - val_precision: 0.8361 - val_recall: 0.6868 - learning_rate: 0.0010
Epoch 2/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8974 - loss: 0.2535 - precision: 0.8640 - recall: 0.9020 - val_accuracy: 0.8081 - val_loss: 0.5021 - val_precision: 0.7463 - val_recall: 0.8242 - learning_rate: 0.0010
Epoch 3/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9571 - loss: 0.1124 - precision: 0.9483 - recall: 0.9517 - val_accuracy: 0.8046 - val_loss: 0.7088 - val_precision: 0.7456 - val_recall: 0.8132 - learning_rate: 0.0010
Epoch 4/20
106/109 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9830 - loss: 0.0569 - precision: 0.9737 - recall: 0.9869
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9831 - loss: 0.0565 - precision: 0.9739 - recall: 

...

217/217 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.5738 - loss: 0.6928 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6920 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5738 - loss: 0.6918 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6910 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5738 - loss: 0.6909 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6901 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5738 - loss: 0.6901 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6892 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


217/217 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.5738 - loss: 0.6928 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6920 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5738 - loss: 0.6918 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6910 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5738 - loss: 0.6909 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6901 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5738 - loss: 0.6901 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.5792 - val_loss: 0.6892 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Config_ID,Embedding,Model,Max_Vocab,Max_Len,Embedding_Dim,LSTM_Units,Dense_Units,Dropout_LSTM,Dropout_Dense,...,L2_Reg,Conv_Filters,Conv_Kernel,Trainable_Embedding,Accuracy,Precision,Recall,F1_Score,Training_Time,Epochs_Trained
0,0,GloVe,LSTM,10000,128,300,64,32,0.3,0.2,...,0.0,NaN,NaN,True,0.823326,0.848684,0.706849,0.771300,17.375926,6
1,0,GloVe,BiLSTM,10000,128,300,64,32,0.3,0.2,...,0.0,NaN,NaN,True,0.819861,0.791086,0.778082,0.784530,19.278354,6
2,0,GloVe,CNN + BiLSTM,10000,128,300,64,32,0.3,0.2,...,0.0,64.0,3.0,True,0.818707,0.756158,0.841096,0.796368,16.922103,6
3,0,FastText,LSTM,10000,128,300,64,32,0.3,0.2,...,0.0,NaN,NaN,True,0.837182,0.791667,0.832877,0.811749,12.581034,6
4,0,FastText,BiLSTM,10000,128,300,64,32,0.3,0.2,...,0.0,NaN,NaN,True,0.839492,0.819209,0.794521,0.806676,19.301407,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,11,FastText,BiLSTM,10000,128,200,256,128,0.6,0.4,...,0.0,NaN,NaN,False,0.578522,0.000000,0.000000,0.000000,93.703944,20
104,11,FastText,CNN + BiLSTM,10000,128,200,256,128,0.6,0.4,...,0.0,64.0,3.0,False,0.578522,0.000000,0.000000,0.000000,58.465503,20
105,11,Word2Vec,LSTM,10000,128,200,256,128,0.6,0.4,...,0.0,NaN,NaN,False,0.578522,0.000000,0.000000,0.000000,50.051320,20
106,11,Word2Vec,BiLSTM,10000,128,200,256,128,0.6,0.4,...,0.0,NaN,NaN,False,0.578522,0.000000,0.000000,0.000000,91.749902,20
